In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pyamg.classical import ruge_stuben_solver

In [ ]:
from topo import SHex
from mesh import Mesh, uniform_nodes_3d
from basis import LagrangeBasisHex
from poisson import poisson_Kloc
from assemble import simple_assembly, simple_build_rhs

In [ ]:
order = 2
L = (1.0,1.0,1.0)
x_max = L[0]
y_max = L[1]
z_max = L[2]

n_elem = 8
vertices, elem_to_vertex, boundary_vertices = \
          uniform_nodes_3d(n_elem,L[0],L[1],L[2])

In [ ]:
topo = SHex()
basis = LagrangeBasisHex(topo, order)
jacb = topo.calc_jacb(vertices[elem_to_vertex])
jacb_det = topo.calc_jacb_det(jacb)
jacb_inv = topo.calc_jacb_inv(jacb)
mesh = Mesh(topo, basis)
mesh.build_mesh(vertices, elem_to_vertex, boundary_vertices)

In [ ]:
Kloc = poisson_Kloc(basis, jacb_det[0], jacb_inv[0])
K = simple_assembly(mesh, Kloc)

def f(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    u = x*(x-x_max)*y*(y-y_max)*z*(z-z_max)
    return u.reshape(shape)

def f2(X):
    shape = X.shape[:-1]
    X = X.reshape((-1,3))
    x = X[:,0]
    y = X[:,1]
    z = X[:,2]
    u = 2*y*(y-y_max)*x*(x-x_max)+\
        2*z*(z-z_max)*x*(x-x_max)+\
        2*y*(y-y_max)*z*(z-z_max)
    return u.reshape(shape)

rhs = simple_build_rhs(topo, basis, mesh, f2)

In [ ]:
plt.spy(K)
np.sum(np.abs((K-K.T).data)>1e-15)

In [ ]:
ml = ruge_stuben_solver(K)
residuals = []
sol = ml.solve(rhs, tol=1e-12, residuals=residuals, maxiter=5000)
len(residuals), residuals[-1]

In [ ]:
dof_phys = mesh.get_dof_phys()

np.max(np.abs(f(dof_phys)-sol))